In [1]:
import pandas as pd
import re

def add_virtual_column(df: pd.DataFrame, role: str, new_column: str) -> pd.DataFrame:
    """Add a computed column to a DataFrame based on a simple expression."""
    valid_label_pattern = re.compile(r'^[A-Za-z_]+$')

    if not valid_label_pattern.match(new_column):
        return pd.DataFrame([])

    role_clean = role.strip()
    if not re.fullmatch(r'[A-Za-z_\+\-\*\s]+', role_clean):
        return pd.DataFrame([])

    columns_in_role = re.findall(r'[A-Za-z_]+', role_clean)
    if any(col not in df.columns for col in columns_in_role):
        return pd.DataFrame([])

    if any(not valid_label_pattern.match(col) for col in df.columns):
        return pd.DataFrame([])

    try:
        df[new_column] = pd.eval(role_clean, local_dict=df.to_dict(orient="series"))
        return df
    except Exception:
        return pd.DataFrame([])


In [2]:
df = pd.DataFrame({
    "quantity": [10, 3],
    "price": [10, 1]
})

result = add_virtual_column(df, "quantity * price", "total")
result


,quantity,price,total
0,10,10,100
1,3,1,3


In [3]:
import pandas as pd

df = pd.DataFrame([[1, 1]] * 2, columns=["label_one", "label_two"])
expected = pd.DataFrame([[1, 1, 2]] * 2, columns=["label_one", "label_two", "label_three"])
assert add_virtual_column(df, "label_one+label_two", "label_three").equals(expected)

df = pd.DataFrame([[1, 1]] * 2, columns=["label_one", "label_two"])
expected = pd.DataFrame([[1, 1, 1]] * 2, columns=["label_one", "label_two", "label_three"])
assert add_virtual_column(df, "label_one * label_two", "label_three").equals(expected)

print("✅ All tests passed!")


✅ All tests passed!
